# Using the BTagging Tool

Getting a tool to work properly is a little tricky. We'll do that here, and then move it into the hints file.

In [1]:
import awkward as ak
import vector
from func_adl_servicex_xaodr25 import FuncADLQueryPHYSLITE
from func_adl_servicex_xaodr25.xAOD.jet_v1 import Jet_v1
from servicex_analysis_utils import to_awk
from servicex import deliver, ServiceXSpec, Sample, dataset
from hist import Hist
import matplotlib.pyplot as plt
import mplhep as hep
import numpy as np
from func_adl import ObjectStream, func_adl_callable
import ast
from typing import Tuple, TypeVar


Setup the code properly!

In [2]:
physlite = FuncADLQueryPHYSLITE()

Now configure the tool to run

- Taking examples from [use of the JetSelectionTool in the data extractor](https://github.com/gordonwatts/sx_training_fetch/blob/main/calratio_training_data/cpp_xaod_utils.py#L123)
- BTaggingSelectionTool info from the [BTagging Docs](https://ftag.docs.cern.ch/calibrations/cdi/interface/btaggingefficiencytool/).
- And from some [example code](https://gitlab.cern.ch/atlas/athena/-/blob/main/PhysicsAnalysis/JetTagging/JetTagPerformanceCalibration/xAODBTaggingEfficiency/util/BTaggingSelectionToolTester.cxx).
- And a better [working example](https://gitlab.cern.ch/atlas/athena/-/blob/main/PhysicsAnalysis/JetTagging/JetTagPerformanceCalibration/xAODBTaggingEfficiency/util/BTaggingToolsExample.cxx)

In [ ]:
cut_name = "FixedCutBEff_77"
tool_name = f"btag_selection_tool_{cut_name}"


query_base = physlite.MetaData(
    {
        "metadata_type": "inject_code",
        "name": f"btag_tool_{tool_name}",
        "header_includes": ["xAODBTaggingEfficiency/BTaggingSelectionTool.h"],
        "private_members": [f"IBTaggingSelectionTool *{tool_name};"],
        "instance_initialization": [
            f'{tool_name}(new BTaggingSelectionTool("{tool_name}"))'
        ],
        "initialize_lines": [
            f'ANA_CHECK(asg::setProperty({tool_name}, "FlvTagCutDefinitionsFileName", "13p6TeV/MC23_2025-06-17_GN2v01_v4.root"));',
            # f'ANA_CHECK(asg::setProperty({tool_name}, "TaggerName", ""));',
            f'ANA_CHECK(asg::setProperty({tool_name}, "OperatingPoint", "{cut_name}"));',
            # f'ANA_CHECK(asg::setProperty({tool_name}, "JetAuthor", "AntiKt4EMPFlowJets"));',  # Default is PFlowJets
            # f'ANA_CHECK(asg::setProperty({tool_name}, "MinPt", 20000));',  # Default is 20 GeV
            f"ANA_CHECK({tool_name}->initialize());",
        ],
        "link_libraries": ["xAODBTaggingEfficiency"],
    }
)

Some other info:

* `13p6TeV/MC23_2025-06-17_GN2v01_v4.root` for Run 3 (MC23 files)
* `13TeV/MC20_2025-06-17_GN2v01_v4.root` for Run 2 (MC20 files)

Possible cut names refer to xx yy zz

Next, define the code that will actually get the tag rate for us.

In [4]:
T = TypeVar("T")

def tag_passed_callback(
    s: ObjectStream[T], a: ast.Call
) -> Tuple[ObjectStream[T], ast.Call]:
    new_s = s.MetaData(
        {
            "metadata_type": "add_cpp_function",
            "name": "jet_is_tagged",
            "code": [f"bool result = static_cast<bool>({tool_name}->accept(*jet))"],
            "result": "result",
            "include_files": [],
            "arguments": ["jet"],
            "return_type": "bool",
        }
    )
    return new_s, a


@func_adl_callable(tag_passed_callback)
def jet_is_tagged(jet: Jet_v1) -> bool:
    """Call the jet selection on the jet.

    * return true or false if the jet passes the selection cut.

    Args:
        jet (Jet_v1): The jet we are operating against
        value_selector (int): Which value (pixel holes, etc.)

    NOTE: This is a dummy function that injects C++ into the object stream to do the
    actual work.

    Returns:
        bool: Is the jet b-tagged?
    """
    ...

Finally, we can actually execute the query!

In [5]:
query = (
    query_base.SelectMany(lambda event: event.Jets())
    .Where(lambda j: j.pt() > 20000 and abs(j.eta()) < 2)
    .Select(
        lambda j: {
            "is_tagged": jet_is_tagged(j),
            "pt": j.pt(),
            "eta": j.eta(),
        }
    )
)

And now we can run on a data sample!

In [6]:
ds_name = "mc23_13p6TeV:mc23_13p6TeV.601237.PhPy8EG_A14_ttbar_hdamp258p75_allhad.deriv.DAOD_PHYSLITE.e8514_s4369_r16083_p6697"

data = to_awk(
    deliver(
        ServiceXSpec(
            Sample=[
                Sample(
                    Name="ttbar_trijet",
                    Dataset=dataset.Rucio(ds_name),
                    NFiles=1,
                    Query=query,  # type: ignore
                )
            ]
        ),
        servicex_name="servicex-release-prod",
    )
)
jets = data["ttbar_trijet"]

Output()

[07/07/25 14:11:05] ERROR    Transform "ttbar_trijet" completed with failures: 1/1 files          ]8;id=895549;file://c:\Users\gordo\Code\llm\atlas-plot-agent\.venv\Lib\site-packages\servicex\query_core.py\query_core.py]8;;\:]8;id=63152;file://c:\Users\gordo\Code\llm\atlas-plot-agent\.venv\Lib\site-packages\servicex\query_core.py#236\236]8;;\
                             failed.Will not cache.                                                                

                    ERROR    A list of failed files is at ]8;id=698385;https://servicex-release-prod.servicex.ssl-hep.org//transformation-request//9575cf6c-41a5-41a9-af36-006dd3b0ac70/results?status=failure\this link]8;;\                               ]8;id=762412;file://c:\Users\gordo\Code\llm\atlas-plot-agent\.venv\Lib\site-packages\servicex\query_core.py\query_core.py]8;;\:]8;id=366622;file://c:\Users\gordo\Code\llm\atlas-plot-agent\.venv\Lib\site-packages\servicex\query_core.py#237\237]8;;\

                    ERROR    Transform Request id: 9575cf6c-41a5-41a9-af36-006dd3b0ac70           ]8;id=68400;file://c:\Users\gordo\Code\llm\atlas-plot-agent\.venv\Lib\site-packages\servicex\query_core.py\query_core.py]8;;\:]8;id=480829;file://c:\Users\gordo\Code\llm\atlas-plot-agent\.venv\Lib\site-packages\servicex\query_core.py#238\238]8;;\

                    ERROR    More information of 'ttbar_trijet' ]8;id=62097;https://atlas-kibana.mwt2.org:5601/s/servicex/app/dashboards?auth_provider_hint=anonymous1#/view/4f197320-f34e-11ed-a6d8-9f6a16cd6d78?embed=true&_g=(time:(from:now-30d%2Fd,to:now))&_a=(filters:!((query:(match_phrase:(requestId:'9575cf6c-41a5-41a9-af36-006dd3b0ac70'))),(query:(match_phrase:(level:'error')))))&show-time-filter=true\HERE]8;;\                              ]8;id=711617;file://c:\Users\gordo\Code\llm\atlas-plot-agent\.venv\Lib\site-packages\servicex\query_core.py\query_core.py]8;;\:]8;id=230720;file://c:\Users\gordo\Code\llm\atlas-plot-agent\.venv\Lib\site-packages\servicex\query_core.py#248\248]8;;\

Exception in callback Query.submit_and_download.<locals>.transform_complete() at 
c:\Users\gordo\Code\llm\atlas-plot-agent\.venv\Lib\site-packages\servicex\query_core.py:195
handle: <Handle Query.submit_and_download.<locals>.transform_complete() at 
c:\Users\gordo\Code\llm\atlas-plot-agent\.venv\Lib\site-packages\servicex\query_core.py:195>
Traceback (most recent call last):
  File "C:\Users\gordo\AppData\Local\Programs\Python\Python313\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gordo\Code\llm\atlas-plot-agent\.venv\Lib\site-packages\servicex\query_core.py", line 253, in 
transform_complete
    raise ServiceXException(errorstr)
servicex.query_core.ServiceXException: Transform "ttbar_trijet" completed with failures: 1/1 files failed.Will not 
cache.

ValueError: ServiceX result path list for ttbar_trijet cannot be empty.